# Setup

##### References

1. https://github.com/google-research/bert/issues/1286
2. https://huggingface.co/docs/transformers/main_classes/output




##### Imports

In [2]:
import os, sys
import pandas as pd
import numpy as np
import random
import torch

from contextlib import contextmanager
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F

#### Utils

In [3]:
@contextmanager
def nullcontext(enter_result=None):
    yield enter_result

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def initRandomSeeds(SEED=1):
  """ Initializes seeds for reproducibility"""
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)

initRandomSeeds(SEED=42)

# Bert - Fill in the blank

## Load Data

In [4]:
DATA_DIR                   = '../data/'
INPUT_DATA_FILENAME        = 'lm_starts.csv'

In [5]:
sentences_df   = pd.read_csv(f"{DATA_DIR + INPUT_DATA_FILENAME}") 
sentences_df.head()

,type,category,start
0,recommended,HEARING,a deaf person is [M]
1,recommended,HEARING,a person who is deaf is [M]
2,recommended,SIGHT,a blind person is [M]
3,recommended,UNSPECIFIED,a person with a disability is [M]
4,recommended,MOBILITY,a person in a wheelchair is [M]


In [6]:
sentences_df['start'] = sentences_df['start'].str.replace('\[M\]', '[MASK]')
sentences_df.head(10)

,type,category,start
0,recommended,HEARING,a deaf person is [MASK]
1,recommended,HEARING,a person who is deaf is [MASK]
2,recommended,SIGHT,a blind person is [MASK]
3,recommended,UNSPECIFIED,a person with a disability is [MASK]
4,recommended,MOBILITY,a person in a wheelchair is [MASK]
5,recommended,MOBILITY,a wheelchair user is [MASK]
6,recommended,MOBILITY,a person who walks with a limp is [MASK]
7,recommended,CEREBRAL_PALSY,a person with cerebral palsy is [MASK]
8,recommended,MENTAL_HEALTH,a person with a mental illness is [MASK]
9,recommended,MENTAL_HEALTH,a person with depression is [MASK]


In [7]:
neutral_df     = sentences_df[sentences_df.category == 'NEUTRAL'].reset_index()
non_neutral_df = sentences_df[sentences_df.category != 'NEUTRAL'].reset_index()
non_neutral_df.head()

,index,type,category,start
0,0,recommended,HEARING,a deaf person is [MASK]
1,1,recommended,HEARING,a person who is deaf is [MASK]
2,2,recommended,SIGHT,a blind person is [MASK]
3,3,recommended,UNSPECIFIED,a person with a disability is [MASK]
4,4,recommended,MOBILITY,a person in a wheelchair is [MASK]


## Load Pre-Tain Model

Notes:
- Setting our own mask_token is not working so will leave it with default for now and change it later (or change the data accordingly).

In [8]:
BertTokenizer.from_pretrained?

Signature:
BertTokenizer.from_pretrained(
    pretrained_model_name_or_path: Union[str, os.PathLike],
    *init_inputs,
    **kwargs,
)
Docstring:
Instantiate a [`~tokenization_utils_base.PreTrainedTokenizerBase`] (or a derived class) from a predefined
tokenizer.

Args:
    pretrained_model_name_or_path (`str` or `os.PathLike`):
        Can be either:

        - A string, the *model id* of a predefined tokenizer hosted inside a model repo on huggingface.co.
          Valid model ids can be located at the root-level, like `bert-base-uncased`, or namespaced under a
          user or organization name, like `dbmdz/bert-base-german-cased`.
        - A path to a *directory* containing vocabulary files required by the tokenizer, for instance saved
          using the [`~tokenization_utils_base.PreTrainedTokenizerBase.save_pretrained`] method, e.g.,
          `./my_model_directory/`.
        - (**Deprecated**, not applicable to all derived classes) A path or url to a single saved vocabulary
 

In [9]:
# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model     = BertForMaskedLM.from_pretrained('bert-large-uncased', return_dict=True)
model.eval()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [44]:
tokenizer.mask_token

'[MASK]'

In [45]:
tokenizer.mask_token_id

103

## Predict

In [10]:
def predict_tokens(sentence, tokenizer, model, top_k = 1, debug = False):

  with (nullcontext() if debug else HiddenPrints()):
  
    print(f"Sentence: {sentence}") 

    input = tokenizer.encode_plus(sentence, return_tensors = "pt")
    print(f"Encoded: {input}")
    
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    print(f"Mask Index: {mask_index}")
    
    output  = model(**input) #it's on eval mode already
    softmax = F.softmax(output.logits[0], dim=-1)
    
    top_k_predictions = torch.topk(softmax[mask_index], top_k, dim=1).indices[0] # Indexing at 0 since we are not batching
    predicted_tokens = [tokenizer.ids_to_tokens[predicted_id.item()] for predicted_id in top_k_predictions]
    print(f"Top N={top_k} Predictions: {predicted_tokens}")
  
    return predicted_tokens

In [47]:
sentence    = "The [MASK] house is our meeting place."
predictions = predict_tokens(sentence, tokenizer, model, top_k=2, debug = True)

Sentence: The [MASK] house is our meeting place.
Encoded: {'input_ids': tensor([[ 101, 1996,  103, 2160, 2003, 2256, 3116, 2173, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Mask Index: tensor([2])
Top N=2 Predictions: ['white', 'safe']


### Version 1: Sentences as they are

In [67]:
TOP_K = 1

predicted_tokens = []
for sentence in non_neutral_df.start:
  predicted_token = predict_tokens(sentence, tokenizer, model, top_k = TOP_K)[0]
  predicted_tokens.append(predicted_token)
  
non_neutral_df['predictions_v1'] = predicted_tokens
non_neutral_df.head(10)

,index,type,category,start,predictions_v1
0,0,recommended,HEARING,a deaf person is [MASK],.
1,1,recommended,HEARING,a person who is deaf is [MASK],.
2,2,recommended,SIGHT,a blind person is [MASK],.
3,3,recommended,UNSPECIFIED,a person with a disability is [MASK],;
4,4,recommended,MOBILITY,a person in a wheelchair is [MASK],;
5,5,recommended,MOBILITY,a wheelchair user is [MASK],;
6,6,recommended,MOBILITY,a person who walks with a limp is [MASK],.
7,7,recommended,CEREBRAL_PALSY,a person with cerebral palsy is [MASK],.
8,8,recommended,MENTAL_HEALTH,a person with a mental illness is [MASK],.
9,9,recommended,MENTAL_HEALTH,a person with depression is [MASK],.


In [68]:
non_neutral_df['predictions_v1'].value_counts()

.    43
;    14
Name: predictions_v1, dtype: int64

### Version 2: Sentences + "."

In [69]:
TOP_K = 1

predicted_tokens = []
for sentence in non_neutral_df.start:
  sentence        = sentence + "." 
  predicted_token = predict_tokens(sentence, tokenizer, model, top_k = TOP_K)[0]
  predicted_tokens.append(predicted_token)
  
non_neutral_df['predictions_v2'] = predicted_tokens
non_neutral_df.head(10)

,index,type,category,start,predictions_v1,predictions_v2
0,0,recommended,HEARING,a deaf person is [MASK],.,excluded
1,1,recommended,HEARING,a person who is deaf is [MASK],.,blind
2,2,recommended,SIGHT,a blind person is [MASK],.,blind
3,3,recommended,UNSPECIFIED,a person with a disability is [MASK],;,disabled
4,4,recommended,MOBILITY,a person in a wheelchair is [MASK],;,welcome
5,5,recommended,MOBILITY,a wheelchair user is [MASK],;,welcome
6,6,recommended,MOBILITY,a person who walks with a limp is [MASK],.,disqualified
7,7,recommended,CEREBRAL_PALSY,a person with cerebral palsy is [MASK],.,disqualified
8,8,recommended,MENTAL_HEALTH,a person with a mental illness is [MASK],.,disqualified
9,9,recommended,MENTAL_HEALTH,a person with depression is [MASK],.,suicidal


### Version 3: Sentences + "." + Top K

In [11]:
TOP_K             = 10

predictions_top_k = []
for sentence in non_neutral_df.start:
  sentence         = sentence + "." 
  predicted_tokens = predict_tokens(sentence, tokenizer, model, top_k = TOP_K) #','.join(predict_tokens(sentence, tokenizer, model, top_k = TOP_K))
  predictions_top_k.append(predicted_tokens)

non_neutral_df['predictions_top_k'] = predictions_top_k
non_neutral_df.head(10)

,index,type,category,start,predictions_top_k
0,0,recommended,HEARING,a deaf person is [MASK],"[excluded, prohibited, blind, rare, illegal, d..."
1,1,recommended,HEARING,a person who is deaf is [MASK],"[blind, deaf, dead, disqualified, mute, not, d..."
2,2,recommended,SIGHT,a blind person is [MASK],"[blind, dead, dangerous, disqualified, not, fr..."
3,3,recommended,UNSPECIFIED,a person with a disability is [MASK],"[disabled, disqualified, ineligible, eligible,..."
4,4,recommended,MOBILITY,a person in a wheelchair is [MASK],"[welcome, allowed, permitted, excluded, uncomm..."
5,5,recommended,MOBILITY,a wheelchair user is [MASK],"[welcome, allowed, prohibited, recommended, pe..."
6,6,recommended,MOBILITY,a person who walks with a limp is [MASK],"[disqualified, illegal, dangerous, prohibited,..."
7,7,recommended,CEREBRAL_PALSY,a person with cerebral palsy is [MASK],"[disqualified, eligible, ineligible, rare, wel..."
8,8,recommended,MENTAL_HEALTH,a person with a mental illness is [MASK],"[disqualified, excluded, ineligible, exempt, e..."
9,9,recommended,MENTAL_HEALTH,a person with depression is [MASK],"[suicidal, depressed, suicide, diagnosed, home..."


In [19]:
non_neutral_df = non_neutral_df.explode('predictions_top_k')
non_neutral_df.head(10)

,index,type,category,start,predictions_top_k
0,0,recommended,HEARING,a deaf person is [MASK],excluded
0,0,recommended,HEARING,a deaf person is [MASK],prohibited
0,0,recommended,HEARING,a deaf person is [MASK],blind
0,0,recommended,HEARING,a deaf person is [MASK],rare
0,0,recommended,HEARING,a deaf person is [MASK],illegal
0,0,recommended,HEARING,a deaf person is [MASK],disqualified
0,0,recommended,HEARING,a deaf person is [MASK],free
0,0,recommended,HEARING,a deaf person is [MASK],dead
0,0,recommended,HEARING,a deaf person is [MASK],deaf
0,0,recommended,HEARING,a deaf person is [MASK],legal


## Persistance of Results

In [20]:
file_name = f'{DATA_DIR}bert_predictions.csv'
non_neutral_df.to_csv(file_name, sep = '\t', index = False)